# Logistic Regression to Classify Terrain by IMU and Odometry Data from TurtleBot3
### By Jacob Laframboise
Logistic regression works great when the data is randomly split into train and test (high 90 accuracy), but it struggles when the data is split into train/test based on which trial number is was collected in. This further supports the hypothesis that data from each run is more similar to itself than data from a given terrain. 



In [23]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.feature_selection import SelectKBest, chi2

import plotly as ply
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib notebook

pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 80)

In [24]:
# load data
dataFolder = r"C:\Users\jaker\Documents\RoboData-Experiment1"
data = "AllDataDFDelta30Squared.csv"

df = pd.read_csv(os.path.join(dataFolder, data))

In [25]:
# fix index from read_csv
df = df.rename(columns={'Unnamed: 0': 'Seq'})
df = df.set_index('Seq')

In [26]:
df.head(10)

,OdomPosX,OdomPosY,OdomOrientZ,OdomOrientW,OdomLinX,OdomAngZ,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,TimeDelta1,TimeDelta1Squared,TimeDelta11,TimeDelta11Squared,TimeDelta21,TimeDelta21Squared,TimeDelta31,TimeDelta31Squared,TimeDelta41,TimeDelta41Squared,TimeDelta51,TimeDelta51Squared,TimeDelta61,TimeDelta61Squared,TimeDelta71,TimeDelta71Squared,TimeDelta81,TimeDelta81Squared,TimeDelta91,TimeDelta91Squared,TimeDelta101,TimeDelta101Squared,TimeDelta111,TimeDelta111Squared,TimeDelta121,TimeDelta121Squared,TimeDelta131,TimeDelta131Squared,TimeDelta141,TimeDelta141Squared,TimeDelta151,TimeDelta151Squared,TimeDelta161,TimeDelta161Squared,TimeDelta171,TimeDelta171Squared,TimeDelta181,TimeDelta181Squared,TimeDelta191,TimeDelta191Squared,TimeDelta201,TimeDelta201Squared,TimeDelta211,TimeDelta211Squared,...,ImuAccelZDelta21Squared,ImuAccelZDelta31,ImuAccelZDelta31Squared,ImuAccelZDelta41,ImuAccelZDelta41Squared,ImuAccelZDelta51,ImuAccelZDelta51Squared,ImuAccelZDelta61,ImuAccelZDelta61Squared,ImuAccelZDelta71,ImuAccelZDelta71Squared,ImuAccelZDelta81,ImuAccelZDelta81Squared,ImuAccelZDelta91,ImuAccelZDelta91Squared,ImuAccelZDelta101,ImuAccelZDelta101Squared,ImuAccelZDelta111,ImuAccelZDelta111Squared,ImuAccelZDelta121,ImuAccelZDelta121Squared,ImuAccelZDelta131,ImuAccelZDelta131Squared,ImuAccelZDelta141,ImuAccelZDelta141Squared,ImuAccelZDelta151,ImuAccelZDelta151Squared,ImuAccelZDelta161,ImuAccelZDelta161Squared,ImuAccelZDelta171,ImuAccelZDelta171Squared,ImuAccelZDelta181,ImuAccelZDelta181Squared,ImuAccelZDelta191,ImuAccelZDelta191Squared,ImuAccelZDelta201,ImuAccelZDelta201Squared,ImuAccelZDelta211,ImuAccelZDelta211Squared,ImuAccelZDelta221,ImuAccelZDelta221Squared,ImuAccelZDelta231,ImuAccelZDelta231Squared,ImuAccelZDelta241,ImuAccelZDelta241Squared,ImuAccelZDelta251,ImuAccelZDelta251Squared,ImuAccelZDelta261,ImuAccelZDelta261Squared,ImuAccelZDelta271,ImuAccelZDelta271Squared,ImuAccelZDelta281,ImuAccelZDelta281Squared,ImuAccelZDelta291,ImuAccelZDelta291Squared,ImuAccelZDelta301,ImuAccelZDelta301Squared,Speed,Terrain,Trial
Seq,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,101.236073,7.727499,0.271481,0.962444,0.148332,0.002560,0.004440,-0.014551,-0.545528,0.837949,0.021284,0.008514,0.013835,0.246004,0.289698,10.174759,0.000030,8.887553e-10,0.056861,0.003233,0.121929,0.014867,0.176428,0.031127,0.237168,0.056249,0.291568,0.085012,0.354699,0.125811,0.415367,0.172530,0.476153,0.226722,0.535056,0.286285,0.593207,0.351895,0.642440,0.412730,0.716158,0.512883,0.776474,0.602912,0.837746,0.701819,0.896430,0.803587,0.959720,0.921063,1.015709,1.031665,1.077910,1.161890,1.131701,1.280748,1.193373,1.424138,1.256298,1.578285,...,0.011351,0.327408,0.107196,0.232238,0.053934,-0.377685,0.142646,0.345364,0.119276,0.051476,0.002650,-0.427364,0.182640,0.128090,0.016407,0.329204,0.108375,-0.207098,0.042890,-0.136469,0.018624,0.096966,0.009402,-0.302866,0.091728,-0.237025,0.056181,0.034118,0.001164,-0.664989,0.442211,-0.645836,0.417104,-0.328604,0.107980,0.043096,0.001857,0.531513,0.282506,0.681150,0.463966,-0.549469,0.301916,-1.055842,1.114803,-0.819415,0.671441,0.315436,0.099500,-0.314239,0.098746,-0.094570,0.008944,-0.463278,0.214626,0.115222,0.013276,15,Turf,2
1,101.236847,7.727973,0.271494,0.962440,0.147744,0.004663,0.004627,-0.015302,-0.545561,0.837913,0.013835,0.008514,0.000532,0.275034,0.313341,9.970354,0.008693,7.556587e-05,0.060903,0.003709,0.118744,0.014100,0.180629,0.032627,0.241271,0.058212,0.295944,0.087583,0.358029,0.128185,0.414178,0.171543,0.474103,0.224774,0.539442,0.290998,0.596471,0.355778,0.651092,0.423920,0.717194,0.514367,0.779053,0.606924,0.831974,0.692180,0.897493,0.805494,0.959977,0.921555,1.019013,1.038388,1.075933,1.157631,1.135894,1.290255,1.197795,1.434714,1.255287,1.575746,...,0.145600,0.090681,0.008223,0.029628,0.000878,-0.582090,0.338829,0.140958,0.019869,-0.522834,0.273355,-0.463876,0.21518

In [27]:
# check for none values and correct size
print(df.isnull().sum().sum()) # check for NaN
df.shape

0


(244905, 1073)

### Feature engineering: elimination

In [28]:
# eliminate columns derived from time, and from angular velocity
columnsToDrop = [x for x in df.columns.tolist() if 'Time' in x or ('Delta' not in x and 'Odom' in x) ]
df = df.drop(columns=columnsToDrop)

In [29]:
df.head(8)

,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,OdomPosXDelta1,OdomPosXDelta1Squared,OdomPosXDelta11,OdomPosXDelta11Squared,OdomPosXDelta21,OdomPosXDelta21Squared,OdomPosXDelta31,OdomPosXDelta31Squared,OdomPosXDelta41,OdomPosXDelta41Squared,OdomPosXDelta51,OdomPosXDelta51Squared,OdomPosXDelta61,OdomPosXDelta61Squared,OdomPosXDelta71,OdomPosXDelta71Squared,OdomPosXDelta81,OdomPosXDelta81Squared,OdomPosXDelta91,OdomPosXDelta91Squared,OdomPosXDelta101,OdomPosXDelta101Squared,OdomPosXDelta111,OdomPosXDelta111Squared,OdomPosXDelta121,OdomPosXDelta121Squared,OdomPosXDelta131,OdomPosXDelta131Squared,OdomPosXDelta141,OdomPosXDelta141Squared,OdomPosXDelta151,OdomPosXDelta151Squared,OdomPosXDelta161,OdomPosXDelta161Squared,OdomPosXDelta171,OdomPosXDelta171Squared,OdomPosXDelta181,OdomPosXDelta181Squared,OdomPosXDelta191,OdomPosXDelta191Squared,OdomPosXDelta201,OdomPosXDelta201Squared,OdomPosXDelta211,OdomPosXDelta211Squared,OdomPosXDelta221,OdomPosXDelta221Squared,OdomPosXDelta231,OdomPosXDelta231Squared,OdomPosXDelta241,OdomPosXDelta241Squared,...,ImuAccelZDelta21Squared,ImuAccelZDelta31,ImuAccelZDelta31Squared,ImuAccelZDelta41,ImuAccelZDelta41Squared,ImuAccelZDelta51,ImuAccelZDelta51Squared,ImuAccelZDelta61,ImuAccelZDelta61Squared,ImuAccelZDelta71,ImuAccelZDelta71Squared,ImuAccelZDelta81,ImuAccelZDelta81Squared,ImuAccelZDelta91,ImuAccelZDelta91Squared,ImuAccelZDelta101,ImuAccelZDelta101Squared,ImuAccelZDelta111,ImuAccelZDelta111Squared,ImuAccelZDelta121,ImuAccelZDelta121Squared,ImuAccelZDelta131,ImuAccelZDelta131Squared,ImuAccelZDelta141,ImuAccelZDelta141Squared,ImuAccelZDelta151,ImuAccelZDelta151Squared,ImuAccelZDelta161,ImuAccelZDelta161Squared,ImuAccelZDelta171,ImuAccelZDelta171Squared,ImuAccelZDelta181,ImuAccelZDelta181Squared,ImuAccelZDelta191,ImuAccelZDelta191Squared,ImuAccelZDelta201,ImuAccelZDelta201Squared,ImuAccelZDelta211,ImuAccelZDelta211Squared,ImuAccelZDelta221,ImuAccelZDelta221Squared,ImuAccelZDelta231,ImuAccelZDelta231Squared,ImuAccelZDelta241,ImuAccelZDelta241Squared,ImuAccelZDelta251,ImuAccelZDelta251Squared,ImuAccelZDelta261,ImuAccelZDelta261Squared,ImuAccelZDelta271,ImuAccelZDelta271Squared,ImuAccelZDelta281,ImuAccelZDelta281Squared,ImuAccelZDelta291,ImuAccelZDelta291Squared,ImuAccelZDelta301,ImuAccelZDelta301Squared,Speed,Terrain,Trial
Seq,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.004440,-0.014551,-0.545528,0.837949,0.021284,0.008514,0.013835,0.246004,0.289698,10.174759,0.000774,5.988262e-07,0.006644,0.000044,0.014467,0.000209,0.022620,0.000512,0.030581,0.000935,0.036799,0.001354,0.045454,0.002066,0.052439,0.002750,0.060308,0.003637,0.069749,0.004865,0.075934,0.005766,0.084594,0.007156,0.091356,0.008346,0.099442,0.009889,0.106577,0.011359,0.114625,0.013139,0.123480,0.015247,0.129635,0.016805,0.137871,0.019008,0.143909,0.020710,0.153248,0.023485,0.159801,0.025536,0.167462,0.028044,0.174639,0.030499,0.182697,0.033378,...,0.011351,0.327408,0.107196,0.232238,0.053934,-0.377685,0.142646,0.345364,0.119276,0.051476,0.002650,-0.427364,0.182640,0.128090,0.016407,0.329204,0.108375,-0.207098,0.042890,-0.136469,0.018624,0.096966,0.009402,-0.302866,0.091728,-0.237025,0.056181,0.034118,0.001164,-0.664989,0.442211,-0.645836,0.417104,-0.328604,0.107980,0.043096,0.001857,0.531513,0.282506,0.681150,0.463966,-0.549469,0.301916,-1.055842,1.114803,-0.819415,0.671441,0.315436,0.099500,-0.314239,0.098746,-0.094570,0.008944,-0.463278,0.214626,0.115222,0.013276,15,Turf,2
1,0.004627,-0.015302,-0.545561,0.837913,0.013835,0.008514,0.000532,0.275034,0.313341,9.970354,0.000774,5.988262e-07,0.007018,0.000049,0.014114,0.000199,0.022786,0.000519,0.030722,0.000944,0.036961,0.001366,0.045614,0.002081,0.051735,0.002677,0.060486,0.003659,0.069903,0.004886,0.076091,0.005790,0.084644,0.007165,0.091133,0.008305,0.099595,0.009919,0.105797,0.011193,0.114792,0.013177,0.123634,0.015285,0.129799,0.016848,0.138039,0.01

### Speed filtering

In [30]:
df = df[df['Speed']==10]

### Logistic Regression
#### Now that we have processed the data, we can apply logistic regression to classify.

In [37]:
# sort the train test split where train on t1 test on t2
dfTrain = df[df['Trial']==2]
dfTest = df[df['Trial']==1]

Y_train = dfTrain['Terrain']
Y_test = dfTest['Terrain']

X_train = dfTrain.drop(columns=['Terrain', 'Speed', 'Trial'])
X_test = dfTest.drop(columns=['Terrain', 'Speed', 'Trial'])

### Feature engineering: Kbest features

In [32]:
featureCount = 30
test = SelectKBest(k=featureCount)
fit = test.fit(X_train, Y_train)
# print(fit.scores_)
X_train = fit.transform(X_train)
X_test = fit.transform(X_test)

In [33]:
# randomly split to train and test
# # separate labels
# terrains = df['Terrain']
# speeds = df['Speed']
# df= df.drop(columns=['Terrain', 'Speed'])

# X = df
# Y = terrains
# X_test, X_train, Y_test, Y_train = train_test_split(X, Y, test_size=0.5)

In [34]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(29354, 30)
(29354,)
(29206, 30)
(29206,)


In [35]:
model = LogisticRegression()
model.fit(X_train, Y_train)

C:\Users\jaker\Anaconda3\envs\tf-p37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\jaker\Anaconda3\envs\tf-p37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
print("Accuracy on test set is: {}".format(model.score(X_test, Y_test)))

Accuracy on test set is: 0.2592275559816476
